# Практическое занятие №2 - локализация объектов на изображениях

1. Разметка данных с помощью виджета QSL для Jupyter (https://github.com/faustomorales/qsl)
2. Расширение выборки за счет модификации размеченных данных (процедура аугоментация).
3. Знакомство с архитектурой нейронных сетей для локализации объектов на изображении.
4. Обучение нейронных сетей для детекции объектов.
5. Проверка обученной модели для обработки видео потока.
6. Трекинг объектов.
7. Распознавание сцен с помощью простого классификатора.
9. Разработка простой системы управления по данным с видео-камеры.

schema1.svg

In [24]:
#@title
%%html
<video controls width="250">
    <source src="https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/video_1.mp4" type="video/mp4"/>
</video>
<video controls width="250">
    <source src="https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/video_3.mp4" type="video/mp4"/>
</video>
<video controls width="250">
    <source src="https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_3/video_4.mp4" type="video/mp4"/>
</video>
<video controls width="250">
    <source src="https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_4/video_4.mp4" type="video/mp4"/>
</video>

# Разметка кадров для обучающей выборки

In [8]:
!pip install -U -q qsl

In [9]:
# Активация расширенных виджетов в Colab (можно закомментировать при запуске в Jupyter)
import google.colab
google.colab.output.enable_custom_widget_manager()

In [10]:
import qsl

In [11]:
%%writefile images.csv
target
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_001.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_002.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_003.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_005.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_006.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_007.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_008.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_009.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_010.jpg
https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_3/image_011.jpg

Overwriting images.csv


In [12]:
import pandas

image_file = pandas.read_csv('images.csv')

In [13]:
labeler = qsl.MediaLabeler(
    items=image_file.to_dict(orient='records'),
    config={
        "regions": [
            {"name": "Object", "multiple": True, "options": [{"name": "bottle"}, {"name": "bug"}]}
        ]
    })
labeler.mode = "dark"
labeler

MediaLabeler(buttons={'prev': False, 'next': True, 'save': True, 'config': True, 'delete': False, 'ignore': Tr…

In [14]:
import json

with open('result.json', 'w') as file:
    file.write(json.dumps(labeler.items, sort_keys=True, indent=4))

In [15]:
labels = None
with open('result.json', 'r') as file:
    labels = file.read()
    print(labels)

[
    {
        "target": "https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_001.jpg"
    },
    {
        "target": "https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_002.jpg"
    },
    {
        "target": "https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_1/image_003.jpg"
    },
    {
        "target": "https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_005.jpg"
    },
    {
        "target": "https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_006.jpg"
    },
    {
        "target": "https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_007.jpg"
    },
    {
        "target": "https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/video_2/image_008.jpg"
    },
    {
        "target": "https://github.com/ant-nik/neural_network_course/raw/main/practice_2_d

In [16]:
!wget 'https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/object_masks.json'

--2023-02-17 05:12:25--  https://github.com/ant-nik/neural_network_course/raw/main/practice_2_data/object_masks.json
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ant-nik/neural_network_course/main/practice_2_data/object_masks.json [following]
--2023-02-17 05:12:25--  https://raw.githubusercontent.com/ant-nik/neural_network_course/main/practice_2_data/object_masks.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 119085 (116K) [text/plain]
Saving to: ‘object_masks.json’

object_masks.json   100%[===================>] 116.29K  --.-KB/s    in 0.002s  

2023-02-17 05:12:25 (45.9 MB/s) - ‘objec

In [17]:
labels = None
with open('object_masks.json', 'r') as file:
    labels = file.read()
label_data = json.loads(labels)

In [18]:
labeler = qsl.MediaLabeler(
    items=label_data,
    
    #image_file.to_dict(orient='records'),
    config={
        "regions": [
            {"name": "Type", "multiple": True, "options": [{"name": "bottle"}, {"name": "bag"}]}
        ]
    })
labeler.labels
labeler.mode = "dark"
labeler

MediaLabeler(buttons={'prev': False, 'next': True, 'save': True, 'config': True, 'delete': True, 'ignore': Fal…

In [19]:
class Struct(object):

    def __init__(self, data):
        for name, value in data.items():
            setattr(self, name, self._wrap(value))

    def _wrap(self, value):
        if isinstance(value, (tuple, list, set, frozenset)): 
            return type(value)([self._wrap(v) for v in value])
        else:
            return Struct(value) if isinstance(value, dict) else value

    def __str__(self):
        result = '{'
        first = True
        for field in self.__dict__.keys():
            if not first:
                result += ', '
            else:
                first = False
            result += f'{field}: {str(self.__dict__[field])}'
        return result + '}'

    def __repr__(self):
        return str(self)

In [20]:
labels = [Struct(label) for label in label_data]
labels[0]

{ignore: False, labels: {boxes: [], dimensions: {height: 914, width: 1628}, image: {}, masks: [], polygons: [{labels: {Object: ['bug']}, points: [{x: 0.642587170887295, y: 0.3516193884040677}, {x: 0.6700005413571787, y: 0.3926353214413227}, {x: 0.7643025357735784, y: 0.45122951149454404}, {x: 0.8235154159885271, y: 0.5117768412162061}, {x: 0.8728594828343176, y: 0.5723241709378682}, {x: 0.9046589925793826, y: 0.5977149866275975}, {x: 0.909045131854564, y: 0.6191995229804453}, {x: 0.934265432686857, y: 0.6133401039751232}, {x: 0.9638718727943313, y: 0.6367777799964117}, {x: 0.9265896889552896, y: 0.7012313890549553}, {x: 0.8893075051162478, y: 0.7656849981134988}, {x: 0.8706664131967269, y: 0.8008415121454316}, {x: 0.8114535329817782, y: 0.8027946518138723}, {x: 0.7686886750487598, y: 0.785216394797906}, {x: 0.7423718393976715, y: 0.7754506964557024}, {x: 0.7335995608473087, y: 0.7344347634184474}, {x: 0.7259238171157413, y: 0.7246690650762438}, {x: 0.7204411430217645, y: 0.697325109718

# Расширение обучающей выборки 

На практике не всегда удается собрать достаточно размеченного материала для качественного обучения нейронной сети. Кроме того, использование большого массива изображений без должного анализа состава приводит к разбалансировке процесса обучения. Например, в общем потоке изображений какие-либо классы объектов могут быть представлены незначительным числом примеров и по этой причине их распознавание во время обучения будет иметь малую значимость на фоне остальных классов. В этой связи все современные модели при обучении предполагают в определенной степени крапотливое конструирование синтетического набора данных на основе исходных изображений. Подробный обзор стратегий расширения выборки за счет изменения исходных изображений представлен в статье "A Comprehensive Survey of Image Augmentation Techniques for Deep Learning" (https://arxiv.org/abs/2205.01491).

Далее будем использовать следующий способ расширения выборки изображений.
1. Извлечение изображений интересующих нас объектов из исходных данных.
2. Создание набора фоновых изображений для размещения объектов.
3. Определение набора трасформаций изображений объектов и фона, включая масштабирование, отражения, повороты, обрезку, изменение цвета/насыщенности, наложение шума.
4. Размещение объектов на фоне в случайных местах.

В настоящее время нет необходимости реализовывать с нуля все шаги синтеза изображений, т.к. существуюет большой набор готовых решений. Наиболее популярными решениями служат следующие пакеты Python.

1. https://pytorch.org/vision/stable/transforms.html
2. https://www.tensorflow.org/tutorials/images/data_augmentation?hl=en
3. https://augmentor.readthedocs.io/en/master/
4. https://github.com/albumentations-team/albumentations

Важным вопросом при расширении выборки является выбор параметров. Он может быть выполнен как эмпирически, так и путем оптимизации. В последнем случае используют либо целевую модель, либо её малый эквивалент с целью вариации параметров и оценки качества обучения. Логичным итогом такого рода работы становятся так называемые "системы автоаугоментации", в которые встраивается универсальная внутренняя модель и алгоритм оптимизации для подбора параметров трансформаций. Они могут использоваться для расширения новых наборов данных без дополнительных усилий со стороны разработчика (но ценой значительного расхода вычислений, т.к. необходимо многократно проводить процесс обучения моделей). Более подробно об основной идее такого автоматического подхода можно почитать в статье сотрудников компании Google (https://arxiv.org/abs/1805.09501), а также изучить похожую реализацию в пакете deepaugment - https://github.com/barisozmen/deepaugment (к сожалению, пакет больше не поддерживается, последнее обновление исходного кода было в 2019 году).
Далее будем использовать более простой "эмпирический способ".

